In [ ]:
!pip install transformers
!pip install konlpy

In [2]:
import pandas as pd
import numpy as np
import pickle

from transformers import BertTokenizerFast, BertModel
from torchtext import transforms as T
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Okt

computer = 'colab'

In [3]:
if computer=='docker':
    train_data_path = '/root/share/data/train.csv'
    test_data_path = '/root/share/data/test.csv'
elif computer=='colab':
    train_data_path = '/content/drive/MyDrive/commit_folder/chungwadae/torch_nlp/data/train.csv'
    test_data_path = '/content/drive/MyDrive/commit_folder/chungwadae/torch_nlp/data/test.csv'

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')
model = BertModel.from_pretrained('kykim/bert-kor-base')

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

train_data = train_data[train_data['data'].notnull()]

In [7]:
train_data.head()

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [14]:
class CustomDataset(Dataset):
  def __init__(self, df, tokenizer):
    self.labels = [label for label in df['category']]
    okt = Okt()
    df['data'] = df['data'].apply(self.tokenizer, tag=okt)
    df['data'] = df['data'].apply(lambda x: ' '.join(x))
    self.sentence = [
        tokenizer(sentence, padding='max_length', max_length=300, truncation=True, return_tensors='pt')
        for sentence in df['data']            
    ]

  def tokenizer(self, sentence, tag):
    return tag.morphs(sentence)

  def __len__(self):
        return len(self.labels)

  def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [16]:
okt = Okt()

In [21]:
def tokenize(sentence, tag):
    return tag.morphs(sentence)

In [27]:
train_data.iloc[1,2]

"학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등 글짓기를 하다 생각했습니다 우리 울산엔 '울산중학교'는 두개입니다 하나는 남중,하나는 여중인데 어째서 우리학교만 '여자'를 붙여야하는가 하고요 남자가 우위였던 때 지어졌던 학교라 그런진 모르겠지만 울산중학교(남중)에도 '남자'를 붙여 울산남자중학교가 되게 해주세요\\n평소에 학교이름을 줄여 부를때에도 우리학교는 울여중,남중은 그냥 울중 이라 부릅니다 몇년동안 그리 불러온, 너무나 익숙해진 이 현실을 이젠 바꿀때가 되지않았나 싶네요,,지금은 조선시대가 아니니까요\\n국내에 이런 학교는 널렸습니다 울산뿐만이 아니라 국내 다른 중/고등학교에 있는 똑같은 문제들을 해결해주셨으면 합니다..."

In [28]:
from tqdm import tqdm

In [30]:
res = []
for i in tqdm(range(len(train_data))):
  res.append(tokenize(train_data.iloc[i,2], okt))

  1%|          | 349/39992 [00:15<29:42, 22.24it/s]


KeyboardInterrupt: ignored

In [22]:
train_data['data'] = train_data['data'].apply(tokenize, tag=okt)

KeyboardInterrupt: ignored

In [15]:
train_dataset = CustomDataset(train_data, tokenizer)

KeyboardInterrupt: ignored